In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## Get base vectors using 3d FFT

In [ ]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters
import numpy as np

In [ ]:
import scipy.ndimage as ndi
from skimage import measure
from sklearn.cluster import DBSCAN, MeanShift
from dataclasses import dataclass

@dataclass
class Cluster:
    i: int
    samples: np.ndarray
    
    @property
    def magnitude(self):
        return np.linalg.norm(self.vector)
    @property
    def confidence(self):
        return self.std.mean()
    @property
    def count(self):
        return len(self.samples)
    @property
    def vector(self):
        return self.samples.mean(axis=0)
    @property
    def std(self):
        return self.samples.std(axis=0)
    @property
    def magnitude(self):
        return np.linalg.norm(self.vector)
    def vector_normalized(self):
        return self.vector / self.magnitude
    
def find_peaks(image, *, threshold: float, min_sigma: float =1.0, max_sigma: float = 2.0):
    """Find peaks in image using difference of gaussian."""
    
    difference = ndi.gaussian_filter(image, min_sigma) - ndi.gaussian_filter(image, max_sigma)
    labels, numlabels = ndi.label(difference > threshold)
    props = measure.regionprops(labels, image)
    peaks = np.array([prop.centroid for prop in props])
    
    return peaks


def find_periodic(peaks) -> dict:
    """Find periodic vectors in list of peaks.
    
    TODO: uniqify vectors (i.e. select one quadrant)
    """
    from scipy.spatial import Delaunay

    D = Delaunay(peaks)

    vertices = D.points
    faces = D.simplices
    # diffs = vertices[faces] - vertices[faces][:,1:2,:]
    diff = (vertices[faces] - vertices[faces][:,0:1,:]).reshape(-1,3)
    
    db = DBSCAN(eps=1.0, min_samples=5).fit(diff)
    core_samples = db.core_sample_indices_
    labels = db.labels_[core_samples]

    core_samples = diff[core_samples]
    
    unique, counts = np.unique(labels, return_counts=True)
    
    d = {}
    for label, count in zip(unique, counts):
        vectors = core_samples[labels==label]
        d[label] = Cluster(i=label, samples=vectors)

    return d

In [ ]:
vol = Volume.load('sample_data.npy')
vol.show_slice()

Use 3D FFT to find regularity in volume.

In [ ]:
fourier = np.fft.fftn(vol.image)
scaled = np.abs(np.fft.fftshift(fourier))
clipped = np.clip(scaled, a_min=0, a_max=1e7)[50:150, 50:150, 50:150]

vol_clipped = Volume(clipped)

vol_clipped.show_slice(index=50, along='y')

Find peaks using difference of gaussians

In [ ]:
peaks = find_peaks(scaled, threshold=1e6)
peaks

Use delaunay triangulation with cluster analysis to find periodic vectors.

In [ ]:
clusters = find_periodic(peaks)
clusters

Plot cluster centers and cluster samples.

In [ ]:
samples = np.vstack([val.samples for val in clusters.values()])
centers = np.array([val.vector for val in clusters.values()])
ORIGIN = np.array([0,0,0]).reshape(1,3)

plotter = pv.PlotterITK()
plotter.add_points(centers)
plotter.add_points(samples)
plotter.add_points(ORIGIN)
plotter.show()

Find similar clusters

In [ ]:
import matplotlib.pyplot as plt

thresh = 0.95

centers_normalized = centers / np.linalg.norm(centers,axis=1).reshape(16,1)
corrmat = np.dot(centers_normalized, centers_normalized.T)

upper_tri = np.triu(corrmat, k=1)
sel = np.abs(upper_tri) > thresh
index = np.argwhere(sel)

plt.imshow(sel)
plt.show()

index

In [ ]:
def merge_similar(clusters, thresh = 0.95):
    centers_normalized = centers / np.linalg.norm(centers,axis=1).reshape(16,1)
    corrmat = np.dot(centers_normalized, centers_normalized.T)

    upper_tri = np.triu(corrmat, k=1)
    sel = np.abs(upper_tri) > thresh
    index = np.argwhere(sel)
    
    for i, j in index:
        c1 = clusters[i]
        c2 = clusters[j]
        assert (c1['magnitude'] - c2['magnitude']) < 0.05
        
    return
 

for i, j in index:
    print(clusters[i])
    print(clusters[j])
    print()

In [ ]:
np.dot(centers_normalized[3], centers_normalized[12])

In [ ]:
np.dot(centers[1], centers[1])

In [ ]:
axes = (
    np.array([0,0,1]),
    np.array([0,1,0]),
    np.array([1,0,0]),
)

In [ ]:
for c in clusters.values():
    v1 = c['vector']
    v2 = axes[2]

    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    
    dot_product = np.dot(v1, v2)
    angle = np.arccos(dot_product)
    
    print(np.degrees(angle))

In [ ]:
x = 0.1

fz = lambda v: v + (np.random.random()-0.5)*x

a = np.array([fz(1), fz(0), fz(0)])
b = np.array([fz(1), fz(0), fz(0)])

print(a)
print(b)

np.dot(a, b)

In [ ]:
a = centers[4] / np.linalg.norm(centers[4])
b = centers[6] / np.linalg.norm(centers[6])

np.dot(a, b)

In [ ]:
a,b